# Brazilian E-Commerce Data Analysis

This notebook provides a comprehensive analysis of the Brazilian E-Commerce dataset from Olist, a Brazilian e-commerce platform. We'll explore customer behavior, sales patterns, geographic distribution, and more to extract actionable business insights.

## 1. Data Loading and Initial Exploration

Let's start by loading the datasets and exploring their structure.


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from mlxtend.frequent_patterns import apriori, association_rules
import folium
from folium.plugins import HeatMap
import warnings
import json
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic


# Suppress warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Display settings for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


In [ ]:
# Load the datasets
try:
    # Adjust these paths to match your local file structure
    orders = pd.read_csv('data/olist_orders_dataset.csv')
    order_items = pd.read_csv('data/olist_order_items_dataset.csv')
    products = pd.read_csv('data/olist_products_dataset.csv')
    customers = pd.read_csv('data/olist_customers_dataset.csv')
    sellers = pd.read_csv('data/olist_sellers_dataset.csv')
    payments = pd.read_csv('data/olist_order_payments_dataset.csv')
    reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
    category_translation = pd.read_csv('data/product_category_name_translation.csv')
    
    print("Data loaded successfully!")
except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please ensure the dataset files are in the correct location.")

# Display basic information about each dataset
datasets = {
    'Orders': orders,
    'Order Items': order_items,
    'Products': products,
    'Customers': customers,
    'Sellers': sellers,
    'Payments': payments,
    'Reviews': reviews
}

for name, df in datasets.items():
    print(f"\n{name} Dataset:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {', '.join(df.columns)}")
    print(f"Missing values: {df.isnull().sum().sum()}")
    print("\nSample data:")
    display(df.head(3))

## 2. Data Preparation and Cleaning

Before diving into analysis, let's clean the data and prepare it for exploration.

In [ ]:
# Convert date columns to datetime
date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date',
                'order_estimated_delivery_date']

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Calculate delivery metrics
orders['delivery_time'] = (orders['order_delivered_customer_date'] - 
                           orders['order_purchase_timestamp']).dt.days

orders['delivery_delay'] = (orders['order_delivered_customer_date'] - 
                            orders['order_estimated_delivery_date']).dt.days

orders['approval_time'] = (orders['order_approved_at'] - 
                          orders['order_purchase_timestamp']).dt.total_seconds() / 3600  # in hours

orders['carrier_time'] = (orders['order_delivered_carrier_date'] - 
                         orders['order_approved_at']).dt.days  # time to reach carrier

# Handle missing values in delivery metrics
orders['delivery_time'] = orders['delivery_time'].fillna(-1)  # -1 indicates not delivered yet
orders['delivery_delay'] = orders['delivery_delay'].fillna(0)  # 0 indicates no delay or not delivered

# Translate product categories to English
products = products.merge(category_translation, on='product_category_name', how='left')
products['product_category_name_english'] = products['product_category_name_english'].fillna('unknown')

# Check for duplicates
for name, df in datasets.items():
    print(f"{name} dataset has {df.duplicated().sum()} duplicate rows")

# Merge datasets for comprehensive analysis
order_details = orders.merge(order_items, on='order_id', how='left')
order_details = order_details.merge(products, on='product_id', how='left')
order_details = order_details.merge(customers, on='customer_id', how='left')
order_details = order_details.merge(sellers, on='seller_id', how='left')
order_details = order_details.merge(payments, on='order_id', how='left')
order_details = order_details.merge(reviews, on='order_id', how='left')

print("Data preparation complete.")
print(f"Comprehensive dataset shape: {order_details.shape}")

## 3. Temporal Analysis

Let's analyze how sales and customer behavior change over time.

In [ ]:
# Extract time components
orders['order_year'] = orders['order_purchase_timestamp'].dt.year
orders['order_month'] = orders['order_purchase_timestamp'].dt.month
orders['order_day'] = orders['order_purchase_timestamp'].dt.day
orders['order_hour'] = orders['order_purchase_timestamp'].dt.hour
orders['order_dayofweek'] = orders['order_purchase_timestamp'].dt.dayofweek
orders['order_quarter'] = orders['order_purchase_timestamp'].dt.quarter
orders['order_yearmonth'] = orders['order_purchase_timestamp'].dt.to_period('M')

# Monthly order trends
monthly_orders = orders.groupby('order_yearmonth').agg({
    'order_id': 'count',
    'delivery_time': 'mean',
    'delivery_delay': 'mean'
}).reset_index()
monthly_orders['order_yearmonth'] = monthly_orders['order_yearmonth'].astype(str)

# Create a time series plot with multiple metrics
fig = make_subplots(rows=2, cols=1, 
                    shared_xaxes=True, 
                    subplot_titles=('Monthly Order Volume', 'Delivery Performance'))

fig.add_trace(
    go.Scatter(x=monthly_orders['order_yearmonth'], y=monthly_orders['order_id'], 
               mode='lines+markers', name='Order Count'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=monthly_orders['order_yearmonth'], y=monthly_orders['delivery_time'], 
               mode='lines+markers', name='Avg Delivery Time (days)'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=monthly_orders['order_yearmonth'], y=monthly_orders['delivery_delay'], 
               mode='lines+markers', name='Avg Delivery Delay (days)'),
    row=2, col=1
)

fig.update_layout(height=800, width=1000, title_text="E-Commerce Trends Over Time")
fig.update_xaxes(tickangle=45)
fig.show()

# Analyze hourly and daily patterns
plt.figure(figsize=(18, 6))

plt.subplot(1, 2, 1)
hourly_orders = orders['order_hour'].value_counts().sort_index()
sns.barplot(x=hourly_orders.index, y=hourly_orders.values, palette='viridis')
plt.title('Orders by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Number of Orders')

plt.subplot(1, 2, 2)
daily_orders = orders['order_dayofweek'].value_counts().sort_index()
sns.barplot(x=daily_orders.index, y=daily_orders.values, palette='viridis')
plt.title('Orders by Day of Week')
plt.xlabel('Day (0=Monday, 6=Sunday)')
plt.ylabel('Number of Orders')
plt.tight_layout()
plt.show()

# Time series decomposition to identify trends, seasonality, and residuals
# First, create a daily time series
daily_orders = orders.groupby(orders['order_purchase_timestamp'].dt.date)['order_id'].count()
daily_orders.index = pd.DatetimeIndex(daily_orders.index)
daily_orders = daily_orders.sort_index()

# Fill missing dates with zeros
idx = pd.date_range(daily_orders.index.min(), daily_orders.index.max())
daily_orders = daily_orders.reindex(idx, fill_value=0)

# Perform time series decomposition
decomposition = seasonal_decompose(daily_orders, model='additive', period=7)  # Weekly seasonality

# Plot the decomposition
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 12))
decomposition.observed.plot(ax=ax1)
ax1.set_title('Observed')
decomposition.trend.plot(ax=ax2)
ax2.set_title('Trend')
decomposition.seasonal.plot(ax=ax3)
ax3.set_title('Seasonality')
decomposition.resid.plot(ax=ax4)
ax4.set_title('Residuals')
plt.tight_layout()
plt.show()

print("Temporal analysis complete.")

## 4. Geographic Analysis

Let's explore the geographic distribution of customers and sellers.

In [ ]:
# Load the GeoJSON file with the structure you provided
brazil_gdf = gpd.read_file('brazil_states.geojson')

# Display the first few rows to confirm the structure
print("GeoJSON structure:")
display(brazil_gdf.head())

# Create a mapping from municipality names to state codes
# This dictionary maps Brazilian municipalities to their respective state codes
# We'll create a more comprehensive mapping based on your data

# First, let's extract unique municipality names from your GeoJSON
municipalities = brazil_gdf['shapeName'].unique()
print(f"Number of municipalities in GeoJSON: {len(municipalities)}")

# Create a function to extract state code from municipality name or other attributes
def extract_state_code(row):
    """
    Extract the state code from municipality data.
    This is a simplified approach - in a real notebook, you might need a more comprehensive mapping.
    """
    # Try to extract from shapeName (some municipalities include state in parentheses)
    name = row['shapeName']
    
    # Check if the name contains state code in parentheses
    if '(' in name and ')' in name:
        possible_state = name.split('(')[1].split(')')[0]
        if len(possible_state) == 2:  # State codes are 2 characters
            return possible_state
    
    # Map major cities directly
    city_to_state = {
        'São Paulo': 'SP',
        'Rio de Janeiro': 'RJ',
        'Brasília': 'DF',
        'Salvador': 'BA',
        'Fortaleza': 'CE',
        'Belo Horizonte': 'MG',
        'Manaus': 'AM',
        'Curitiba': 'PR',
        'Recife': 'PE',
        'Porto Alegre': 'RS',
        'Belém': 'PA',
        'Goiânia': 'GO',
        # Add more mappings as needed
    }
    
    if name in city_to_state:
        return city_to_state[name]
    
    # Try to extract from shapeID or other attributes
    # This would depend on the specific structure of your data
    
    # Default to None if we can't determine the state
    return None

# Apply the function to add a state column
brazil_gdf['state_code'] = brazil_gdf.apply(extract_state_code, axis=1)

# Check how many municipalities were mapped to states
mapped_count = brazil_gdf['state_code'].notna().sum()
print(f"Municipalities mapped to states: {mapped_count} out of {len(brazil_gdf)}")

# For municipalities without a state code, we can try to infer from geographic proximity
# This is a simplified approach - in a real notebook, you would use a more comprehensive method

# Now let's prepare the e-commerce data for geographic analysis
# Fix the merge issue: customers → orders → order_items

# First, merge orders with customers to get customer location data
customer_orders = orders.merge(customers, on='customer_id')

# Then, merge with order_items to get product and seller information
geographic_data = customer_orders.merge(order_items, on='order_id')

# Finally, merge with sellers to get seller location data
geographic_data = geographic_data.merge(sellers, on='seller_id')

# Check the resulting dataframe
print("Geographic data shape:", geographic_data.shape)
print("\nSample of geographic data:")
display(geographic_data[['customer_id', 'customer_state', 'seller_id', 'seller_state', 'order_id', 'product_id']].head())

# Count orders by customer state
state_orders = geographic_data.groupby('customer_state').size().reset_index(name='order_count')
print("\nOrder counts by state:")
display(state_orders)

# Now we need to create a state-level GeoDataFrame for our choropleth map
# Since your GeoJSON is at the municipality level, we'll need to aggregate to states

# First, let's try to identify which municipalities belong to which states
# We can use the state_code column we created earlier

# For municipalities that we couldn't map automatically, we might need manual mapping
# or we could use a spatial join with a state-level GeoJSON if available

# Let's create a state-level GeoDataFrame by dissolving municipalities
# We'll only use municipalities that we've successfully mapped to states
brazil_gdf_with_state = brazil_gdf[brazil_gdf['state_code'].notna()].copy()

# If we don't have enough municipalities mapped to states, we might need an alternative approach
if len(brazil_gdf_with_state) < len(brazil_gdf) * 0.5:
    print("Warning: Less than 50% of municipalities mapped to states.")
    print("Using an alternative approach with state centroids.")
    
    # Create a GeoDataFrame with state centroids
    # This is a fallback if we can't create proper state polygons
    
    # Dictionary of Brazilian state centroids (approximate coordinates)
    state_coordinates = {
        'AC': (-9.0238, -70.812),   # Acre
        'AL': (-9.5713, -36.782),   # Alagoas
        'AM': (-3.4168, -65.8561),  # Amazonas
        'AP': (1.4035, -51.7963),   # Amapá
        'BA': (-12.9718, -41.7007), # Bahia
        'CE': (-5.4984, -39.3206),  # Ceará
        'DF': (-15.7998, -47.8645), # Distrito Federal
        'ES': (-19.1834, -40.3089), # Espírito Santo
        'GO': (-15.827, -49.8362),  # Goiás
        'MA': (-5.7945, -45.7445),  # Maranhão
        'MG': (-18.5122, -44.555),  # Minas Gerais
        'MS': (-20.7722, -54.7852), # Mato Grosso do Sul
        'MT': (-12.6819, -56.9211), # Mato Grosso
        'PA': (-3.9784, -53.3306),  # Pará
        'PB': (-7.1219, -36.7272),  # Paraíba
        'PE': (-8.8137, -36.9541),  # Pernambuco
        'PI': (-7.7183, -42.7289),  # Piauí
        'PR': (-24.8951, -51.6584), # Paraná
        'RJ': (-22.9099, -43.2095), # Rio de Janeiro
        'RN': (-5.4026, -36.9541),  # Rio Grande do Norte
        'RO': (-10.83, -63.34),     # Rondônia
        'RR': (2.7376, -62.0751),   # Roraima
        'RS': (-30.0346, -51.2177), # Rio Grande do Sul
        'SC': (-27.2423, -50.2189), # Santa Catarina
        'SE': (-10.5741, -37.3857), # Sergipe
        'SP': (-23.5505, -46.6333), # São Paulo
        'TO': (-10.1753, -48.2982)  # Tocantins
    }
    
    # Create a GeoDataFrame with state centroids
    state_points = pd.DataFrame(
        [(state, coords[0], coords[1]) for state, coords in state_coordinates.items()],
        columns=['state_code', 'latitude', 'longitude']
    )
    
    # Convert to GeoDataFrame
    geometry = [Point(xy) for xy in zip(state_points['longitude'], state_points['latitude'])]
    state_gdf = gpd.GeoDataFrame(state_points, geometry=geometry, crs="EPSG:4326")
    
    # Merge with order counts
    state_gdf = state_gdf.merge(state_orders, left_on='state_code', right_on='customer_state', how='left')
    state_gdf['order_count'] = state_gdf['order_count'].fillna(0)
    
    # Create a point-based map
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    
    # Plot Brazil's municipalities as background
    brazil_gdf.plot(ax=ax, color='lightgray', edgecolor='gray', linewidth=0.1)
    
    # Plot state points with size proportional to order count
    state_gdf.plot(
        ax=ax,
        column='order_count',
        markersize=state_gdf['order_count'] / state_gdf['order_count'].max() * 500 + 50,
        cmap='viridis',
        legend=True,
        legend_kwds={'label': 'Number of Orders', 'orientation': 'horizontal'}
    )
    
    # Add state labels
    for idx, row in state_gdf.iterrows():
        ax.text(
            row.geometry.x,
            row.geometry.y,
            f"{row['state_code']}\n{int(row['order_count'])}",
            fontsize=8,
            ha='center',
            va='center',
            color='white',
            fontweight='bold'
        )
    
    plt.title('Distribution of Orders Across Brazilian States', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
else:
    # If we have enough municipalities mapped to states, we can create proper state polygons
    # Group by state and dissolve geometries
    state_gdf = brazil_gdf_with_state.dissolve(by='state_code', aggfunc='sum')
    state_gdf = state_gdf.reset_index()
    
    # Merge with order counts
    state_gdf = state_gdf.merge(state_orders, left_on='state_code', right_on='customer_state', how='left')
    state_gdf['order_count'] = state_gdf['order_count'].fillna(0)
    
    # Create the choropleth map
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    
    # Plot the map with a color gradient based on order count
    state_gdf.plot(
        column='order_count',
        ax=ax,
        legend=True,
        cmap='viridis',
        edgecolor='black',
        linewidth=0.5,
        legend_kwds={'label': 'Number of Orders', 'orientation': 'horizontal'}
    )
    
    # Add state labels
    for idx, row in state_gdf.iterrows():
        # Get the centroid of each state polygon
        centroid = row['geometry'].centroid
        # Add the state code and order count as text
        ax.text(
            centroid.x, 
            centroid.y, 
            f"{row['state_code']}\n{int(row['order_count'])}",
            fontsize=8,
            ha='center',
            va='center',
            color='white',
            fontweight='bold'
        )
    
    plt.title('Distribution of Orders Across Brazilian States', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Now let's analyze customer-seller distances
# Add coordinates to the geographic data
geographic_data['customer_lat'] = geographic_data['customer_state'].map(lambda x: state_coordinates.get(x, (np.nan, np.nan))[0])
geographic_data['customer_lon'] = geographic_data['customer_state'].map(lambda x: state_coordinates.get(x, (np.nan, np.nan))[1])
geographic_data['seller_lat'] = geographic_data['seller_state'].map(lambda x: state_coordinates.get(x, (np.nan, np.nan))[0])
geographic_data['seller_lon'] = geographic_data['seller_state'].map(lambda x: state_coordinates.get(x, (np.nan, np.nan))[1])

# Calculate the distance between customer and seller using geodesic distance (in km)
def calculate_distance(row):
    if pd.isna(row['customer_lat']) or pd.isna(row['seller_lat']):
        return np.nan
    
    customer_coords = (row['customer_lat'], row['customer_lon'])
    seller_coords = (row['seller_lat'], row['seller_lon'])
    
    return geodesic(customer_coords, seller_coords).kilometers

# Apply the distance calculation
geographic_data['distance_km'] = geographic_data.apply(calculate_distance, axis=1)

# Analyze the distribution of distances
plt.figure(figsize=(12, 6))
sns.histplot(geographic_data['distance_km'].dropna(), bins=50, kde=True)
plt.title('Distribution of Customer-Seller Distances')
plt.xlabel('Distance (km)')
plt.ylabel('Frequency')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Calculate average distance by product category
product_distances = geographic_data.merge(products[['product_id', 'product_category_name']], on='product_id')
category_distances = product_distances.groupby('product_category_name')['distance_km'].agg(['mean', 'median', 'count']).reset_index()
category_distances = category_distances.sort_values('mean', ascending=False)

# Display top and bottom categories by average distance
print("\nProduct categories with highest average customer-seller distance:")
display(category_distances.head(10))

print("\nProduct categories with lowest average customer-seller distance:")
display(category_distances.tail(10))

# Visualize top 15 categories by average distance
plt.figure(figsize=(14, 8))
top_categories = category_distances.head(15)
sns.barplot(x='mean', y='product_category_name', data=top_categories, palette='viridis')
plt.title('Product Categories with Highest Average Customer-Seller Distance')
plt.xlabel('Average Distance (km)')
plt.ylabel('Product Category')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Create a flow map showing the volume of transactions between states
# Count transactions between states
state_flows = geographic_data.groupby(['customer_state', 'seller_state']).size().reset_index(name='transaction_count')
state_flows = state_flows.sort_values('transaction_count', ascending=False)

print("\nTop state-to-state transaction flows:")
display(state_flows.head(15))

# Create a network visualization of the top flows
top_flows = state_flows[state_flows['transaction_count'] > 100].copy()

# Create a Brazil map as background
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# Plot Brazil's municipalities as background
brazil_gdf.plot(ax=ax, color='lightgray', edgecolor='gray', linewidth=0.1)

# Add state labels at centroids
for state_code, coords in state_coordinates.items():
    ax.text(
        coords[1],  # longitude
        coords[0],  # latitude
        state_code,
        fontsize=8,
        ha='center',
        va='center',
        fontweight='bold'
    )

# Draw lines between states for top flows
for idx, flow in top_flows.iterrows():
    customer_state = flow['customer_state']
    seller_state = flow['seller_state']
    
    if customer_state in state_coordinates and seller_state in state_coordinates:
        # Get coordinates
        customer_coords = state_coordinates[customer_state]
        seller_coords = state_coordinates[seller_state]
        
        # Draw line with width proportional to transaction count
        line_width = 0.5 + (flow['transaction_count'] / top_flows['transaction_count'].max() * 5)
        
        # Draw the line
        ax.plot(
            [customer_coords[1], seller_coords[1]],  # longitude
            [customer_coords[0], seller_coords[0]],  # latitude
            color='blue',
            alpha=0.5,
            linewidth=line_width
        )

plt.title('Major E-Commerce Transaction Flows Between Brazilian States', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

# Analyze delivery time by distance
# Ensure we have delivery time data
if 'order_delivered_customer_date' in geographic_data.columns and 'order_purchase_timestamp' in geographic_data.columns:
    # Convert date columns to datetime
    geographic_data['order_delivered_customer_date'] = pd.to_datetime(geographic_data['order_delivered_customer_date'])
    geographic_data['order_purchase_timestamp'] = pd.to_datetime(geographic_data['order_purchase_timestamp'])
    
    # Calculate delivery time in days
    geographic_data['delivery_time_days'] = (geographic_data['order_delivered_customer_date'] - 
                                           geographic_data['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)
    
    # Remove outliers and invalid values
    valid_delivery = geographic_data[
        (geographic_data['delivery_time_days'] > 0) & 
        (geographic_data['delivery_time_days'] < 100) &  # Remove extreme outliers
        (geographic_data['distance_km'].notna())
    ]
    
    # Create distance bins
    valid_delivery['distance_bin'] = pd.cut(
        valid_delivery['distance_km'],
        bins=[0, 200, 500, 1000, 1500, 2000, 3000, 5000],
        labels=['0-200', '200-500', '500-1000', '1000-1500', '1500-2000', '2000-3000', '3000+']
    )
    
    # Calculate average delivery time by distance bin
    delivery_by_distance = valid_delivery.groupby('distance_bin')['delivery_time_days'].agg(
        ['mean', 'median', 'std', 'count']
    ).reset_index()
    
    print("\nDelivery time by distance:")
    display(delivery_by_distance)
    
    # Visualize relationship between distance and delivery time
    plt.figure(figsize=(14, 8))
    
    # Boxplot
    plt.subplot(1, 2, 1)
    sns.boxplot(x='distance_bin', y='delivery_time_days', data=valid_delivery)
    plt.title('Delivery Time by Distance')
    plt.xlabel('Distance (km)')
    plt.ylabel('Delivery Time (days)')
    plt.xticks(rotation=45)
    
    # Scatter plot with trend line
    plt.subplot(1, 2, 2)
    sns.scatterplot(x='distance_km', y='delivery_time_days', data=valid_delivery, alpha=0.3)
    sns.regplot(x='distance_km', y='delivery_time_days', data=valid_delivery, scatter=False, color='red')
    plt.title('Delivery Time vs. Distance')
    plt.xlabel('Distance (km)')
    plt.ylabel('Delivery Time (days)')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate correlation
    correlation = valid_delivery['distance_km'].corr(valid_delivery['delivery_time_days'])
    print(f"\nCorrelation between distance and delivery time: {correlation:.4f}")
    
    # Analyze delivery efficiency by state
    state_efficiency = valid_delivery.groupby('seller_state').agg({
        'delivery_time_days': ['mean', 'median'],
        'distance_km': 'mean',
        'order_id': 'count'
    }).reset_index()
    
    # Flatten multi-level columns
    state_efficiency.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in state_efficiency.columns]
    
    # Calculate delivery efficiency (days per 1000 km)
    state_efficiency['days_per_1000km'] = state_efficiency['delivery_time_days_mean'] / (state_efficiency['distance_km_mean'] / 1000)
    
    # Sort by efficiency (lower is better)
    state_efficiency = state_efficiency.sort_values('days_per_1000km')
    
    print("\nDelivery efficiency by seller state:")
    display(state_efficiency)
    
    # Visualize delivery efficiency by state
    plt.figure(figsize=(14, 8))
    sns.barplot(x='seller_state', y='days_per_1000km', data=state_efficiency)
    plt.title('Delivery Efficiency by Seller State (Days per 1000 km)')
    plt.xlabel('Seller State')
    plt.ylabel('Days per 1000 km')
    plt.xticks(rotation=90)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

print("\nGeographic analysis complete.")

## 5. Customer Segmentation with RFM Analysis

Let's segment customers using the RFM (Recency, Frequency, Monetary) framework.

In [ ]:
# Calculate RFM metrics
# Set the analysis date as the day after the last order
last_order_date = orders['order_purchase_timestamp'].max()
analysis_date = last_order_date + timedelta(days=1)

# Get order values
order_values = payments.groupby('order_id')['payment_value'].sum().reset_index()
orders_with_values = orders.merge(order_values, on='order_id', how='left')

# Calculate customer-level RFM metrics
rfm = orders_with_values.groupby('customer_id').agg({
    'order_purchase_timestamp': lambda x: (analysis_date - x.max()).days,  # Recency
    'order_id': 'count',  # Frequency
    'payment_value': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

# Create RFM segments
rfm['r_score'] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])  # 5 is most recent
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])  # 5 is most frequent
rfm['m_score'] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])  # 5 is highest monetary value

# Calculate RFM score
rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)

# Define customer segments
def segment_customer(row):
    r, f, m = int(row['r_score']), int(row['f_score']), int(row['m_score'])
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Loyal Customers'
    elif r >= 3 and f >= 1 and m >= 2:
        return 'Potential Loyalists'
    elif r >= 4 and f <= 2 and m <= 2:
        return 'New Customers'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    elif r <= 2 and f >= 2 and m >= 2:
        return 'Needs Attention'
    elif r <= 1 and f <= 2 and m <= 2:
        return 'Lost'
    else:
        return 'Others'

rfm['segment'] = rfm.apply(segment_customer, axis=1)

# Visualize customer segments
segment_counts = rfm['segment'].value_counts().reset_index()
segment_counts.columns = ['segment', 'count']

plt.figure(figsize=(12, 6))
sns.barplot(x='segment', y='count', data=segment_counts, palette='viridis')
plt.title('Customer Segments')
plt.xlabel('Segment')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Analyze segment characteristics
segment_analysis = rfm.groupby('segment').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean',
    'customer_id': 'count'
}).reset_index()

segment_analysis.columns = ['segment', 'avg_recency', 'avg_frequency', 'avg_monetary', 'customer_count']
segment_analysis['avg_order_value'] = segment_analysis['avg_monetary'] / segment_analysis['avg_frequency']

print("Customer segment analysis:")
display(segment_analysis.sort_values('customer_count', ascending=False))

# Visualize segment characteristics with a radar chart
segments = segment_analysis['segment'].tolist()
metrics = ['avg_recency', 'avg_frequency', 'avg_monetary', 'avg_order_value']

# Normalize the metrics for radar chart
scaler = StandardScaler()
segment_analysis_scaled = segment_analysis.copy()
segment_analysis_scaled[metrics] = scaler.fit_transform(segment_analysis[metrics])

# Create radar chart
fig = go.Figure()

for i, segment in enumerate(segments):
    values = segment_analysis_scaled[segment_analysis_scaled['segment'] == segment][metrics].values.flatten().tolist()
    values += [values[0]]  # Close the loop
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=metrics + [metrics[0]],  # Close the loop
        fill='toself',
        name=segment
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
        )
    ),
    title="Customer Segment Characteristics",
    showlegend=True
)

fig.show()

print("Customer segmentation complete.")

## 6. Product Analysis

Let's analyze product categories, pricing, and popularity.

In [ ]:
# Analyze product categories
product_category_counts = products['product_category_name_english'].value_counts().reset_index()
product_category_counts.columns = ['category', 'product_count']

# Get sales by category
category_sales = order_items.merge(products, on='product_id').groupby('product_category_name_english').agg({
    'order_id': 'count',
    'price': 'sum'
}).reset_index()
category_sales.columns = ['category', 'order_count', 'total_sales']
category_sales['average_price'] = category_sales['total_sales'] / category_sales['order_count']

# Merge product counts with sales data
category_analysis = product_category_counts.merge(category_sales, on='category', how='left')
category_analysis = category_analysis.fillna(0)
category_analysis['sales_per_product'] = category_analysis['order_count'] / category_analysis['product_count']
category_analysis = category_analysis.sort_values('total_sales', ascending=False)

# Visualize top categories by sales
plt.figure(figsize=(14, 8))
sns.barplot(x='total_sales', y='category', data=category_analysis.head(15), palette='viridis')
plt.title('Top 15 Categories by Sales')
plt.xlabel('Total Sales (BRL)')
plt.ylabel('Category')
plt.tight_layout()
plt.show()

# Create a scatter plot of product count vs sales
plt.figure(figsize=(12, 8))
plt.scatter(
    category_analysis['product_count'], 
    category_analysis['total_sales'],
    s=category_analysis['average_price'] * 10,  # Size represents average price
    alpha=0.6
)

# Add labels for top categories
for i, row in category_analysis.head(10).iterrows():
    plt.annotate(
        row['category'],
        xy=(row['product_count'], row['total_sales']),
        xytext=(5, 5),
        textcoords='offset points'
    )

plt.title('Product Count vs Sales by Category')
plt.xlabel('Number of Products')
plt.ylabel('Total Sales (BRL)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Price distribution analysis
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(order_items['price'], bins=50, kde=True)
plt.title('Price Distribution')
plt.xlabel('Price (BRL)')
plt.ylabel('Frequency')
plt.xlim(0, order_items['price'].quantile(0.95))  # Limit to 95th percentile to handle outliers

plt.subplot(1, 2, 2)
sns.boxplot(y='price', data=order_items)
plt.title('Price Boxplot')
plt.ylabel('Price (BRL)')
plt.ylim(0, order_items['price'].quantile(0.95))  # Limit to 95th percentile to handle outliers

plt.tight_layout()
plt.show()

# Product association analysis (market basket analysis)
# Create a one-hot encoded matrix of products purchased together
# First, get products purchased in each order
order_products = order_items[['order_id', 'product_id']].drop_duplicates()
order_products = order_products.merge(products[['product_id', 'product_category_name_english']], on='product_id')

# Create a pivot table for one-hot encoding
basket = order_products.pivot_table(
    index='order_id',
    columns='product_category_name_english',
    values='product_id',
    aggfunc='count',
    fill_value=0
)

# Convert to binary (purchased or not)
basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

# Generate frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

# Generate association rules
if not frequent_itemsets.empty:
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    
    # Display top association rules
    if not rules.empty:
        print("Top product association rules:")
        display(rules.sort_values('lift', ascending=False).head(10))
        
        # Visualize top associations
        plt.figure(figsize=(10, 8))
        plt.scatter(rules['support'], rules['confidence'], alpha=0.5, s=rules['lift']*20)
        
        # Add labels for top rules
        for i, rule in rules.sort_values('lift', ascending=False).head(5).iterrows():
            plt.annotate(
                f"{list(rule['antecedents'])[0]} → {list(rule['consequents'])[0]}",
                xy=(rule['support'], rule['confidence']),
                xytext=(5, 5),
                textcoords='offset points'
            )
        
        plt.title('Association Rules - Support vs Confidence')
        plt.xlabel('Support')
        plt.ylabel('Confidence')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    else:
        print("No association rules found with the given thresholds.")
else:
    print("No frequent itemsets found with the given support threshold.")

print("Product analysis complete.")

## 7. Delivery and Logistics Analysis

Let's analyze delivery performance and its impact on customer satisfaction.

In [ ]:
# Filter for delivered orders only
delivered_orders = orders[orders['order_status'] == 'delivered'].copy()

# Calculate delivery performance metrics
delivery_stats = delivered_orders.agg({
    'delivery_time': ['mean', 'median', 'min', 'max', 'std'],
    'delivery_delay': ['mean', 'median', 'min', 'max', 'std']
})

print("Delivery performance statistics:")
display(delivery_stats)

# Analyze delivery time distribution
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(delivered_orders['delivery_time'], bins=30, kde=True)
plt.title('Delivery Time Distribution')
plt.xlabel('Delivery Time (days)')
plt.ylabel('Frequency')
plt.axvline(delivered_orders['delivery_time'].mean(), color='red', linestyle='--', label=f'Mean: {delivered_orders["delivery_time"].mean():.1f} days')
plt.legend()

plt.subplot(1, 2, 2)
sns.histplot(delivered_orders['delivery_delay'], bins=30, kde=True)
plt.title('Delivery Delay Distribution')
plt.xlabel('Delivery Delay (days)')
plt.ylabel('Frequency')
plt.axvline(0, color='green', linestyle='--', label='On Time')
plt.axvline(delivered_orders['delivery_delay'].mean(), color='red', linestyle='--', label=f'Mean: {delivered_orders["delivery_delay"].mean():.1f} days')
plt.legend()

plt.tight_layout()
plt.show()

# Analyze relationship between delivery performance and customer satisfaction
delivery_reviews = delivered_orders.merge(reviews[['order_id', 'review_score']], on='order_id')

# Group by delivery time buckets and calculate average review score
delivery_time_bins = [0, 7, 14, 21, 28, float('inf')]
delivery_time_labels = ['0-7 days', '8-14 days', '15-21 days', '22-28 days', '29+ days']
delivery_reviews['delivery_time_bucket'] = pd.cut(delivery_reviews['delivery_time'], bins=delivery_time_bins, labels=delivery_time_labels)

delivery_time_satisfaction = delivery_reviews.groupby('delivery_time_bucket').agg({
    'review_score': 'mean',
    'order_id': 'count'
}).reset_index()
delivery_time_satisfaction.columns = ['delivery_time_bucket', 'avg_review_score', 'order_count']

# Group by delivery delay buckets and calculate average review score
delivery_delay_bins = [-float('inf'), -7, -3, 0, 3, 7, float('inf')]
delivery_delay_labels = ['7+ days early', '3-7 days early', '0-3 days early', '0-3 days late', '3-7 days late', '7+ days late']
delivery_reviews['delivery_delay_bucket'] = pd.cut(delivery_reviews['delivery_delay'], bins=delivery_delay_bins, labels=delivery_delay_labels)

delivery_delay_satisfaction = delivery_reviews.groupby('delivery_delay_bucket').agg({
    'review_score': 'mean',
    'order_id': 'count'
}).reset_index()
delivery_delay_satisfaction.columns = ['delivery_delay_bucket', 'avg_review_score', 'order_count']

# Visualize the relationship
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.barplot(x='delivery_time_bucket', y='avg_review_score', data=delivery_time_satisfaction, palette='viridis')
plt.title('Average Review Score by Delivery Time')
plt.xlabel('Delivery Time')
plt.ylabel('Average Review Score')
plt.ylim(1, 5)
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
sns.barplot(x='delivery_delay_bucket', y='avg_review_score', data=delivery_delay_satisfaction, palette='viridis')
plt.title('Average Review Score by Delivery Delay')
plt.xlabel('Delivery Delay')
plt.ylabel('Average Review Score')
plt.ylim(1, 5)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Analyze delivery performance by state
state_delivery = delivered_orders.merge(customers[['customer_id', 'customer_state']], on='customer_id')
state_delivery_perf = state_delivery.groupby('customer_state').agg({
    'delivery_time': 'mean',
    'delivery_delay': 'mean',
    'order_id': 'count'
}).reset_index()
state_delivery_perf.columns = ['state', 'avg_delivery_time', 'avg_delivery_delay', 'order_count']
state_delivery_perf = state_delivery_perf.sort_values('order_count', ascending=False)

# Visualize delivery performance by state
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
sns.barplot(x='state', y='avg_delivery_time', data=state_delivery_perf.head(10), palette='viridis')
plt.title('Average Delivery Time by State (Top 10 by Order Count)')
plt.xlabel('State')
plt.ylabel('Average Delivery Time (days)')
plt.xticks(rotation=0)

plt.subplot(2, 1, 2)
sns.barplot(x='state', y='avg_delivery_delay', data=state_delivery_perf.head(10), palette='viridis')
plt.title('Average Delivery Delay by State (Top 10 by Order Count)')
plt.xlabel('State')
plt.ylabel('Average Delivery Delay (days)')
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

print("Delivery and logistics analysis complete.")

## 8. Customer Satisfaction and Review Analysis

Let's analyze customer reviews and satisfaction levels.

In [ ]:
# Review score distribution
review_score_counts = reviews['review_score'].value_counts().sort_index().reset_index()
review_score_counts.columns = ['score', 'count']

# Calculate review statistics
review_stats = reviews['review_score'].describe()
print("Review score statistics:")
display(review_stats)

# Visualize review score distribution
plt.figure(figsize=(10, 6))
sns.barplot(x='score', y='count', data=review_score_counts, palette='viridis')
plt.title('Distribution of Review Scores')
plt.xlabel('Review Score')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

# Calculate Net Promoter Score (NPS)
# Promoters: 5-star reviews
# Passives: 4-star reviews
# Detractors: 1-3 star reviews
promoters = reviews[reviews['review_score'] == 5].shape[0]
passives = reviews[reviews['review_score'] == 4].shape[0]
detractors = reviews[reviews['review_score'] <= 3].shape[0]
total_reviews = reviews.shape[0]

nps = (promoters - detractors) / total_reviews * 100

print(f"Net Promoter Score (NPS): {nps:.2f}%")

# Analyze review comment length
reviews['comment_length'] = reviews['review_comment_message'].fillna('').apply(len)

# Visualize comment length by review score
plt.figure(figsize=(10, 6))
sns.boxplot(x='review_score', y='comment_length', data=reviews)
plt.title('Review Comment Length by Score')
plt.xlabel('Review Score')
plt.ylabel('Comment Length (characters)')
plt.tight_layout()
plt.show()

# Analyze time to review
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])

# Merge with orders to get purchase date
review_timing = reviews.merge(orders[['order_id', 'order_purchase_timestamp']], on='order_id')
review_timing['days_to_review'] = (review_timing['review_creation_date'] - review_timing['order_purchase_timestamp']).dt.days

# Visualize time to review by score
plt.figure(figsize=(10, 6))
sns.boxplot(x='review_score', y='days_to_review', data=review_timing)
plt.title('Days to Review by Score')
plt.xlabel('Review Score')
plt.ylabel('Days from Purchase to Review')
plt.ylim(0, review_timing['days_to_review'].quantile(0.95))  # Limit to 95th percentile to handle outliers
plt.tight_layout()
plt.show()

# Analyze relationship between product categories and review scores
category_reviews = order_items.merge(products[['product_id', 'product_category_name_english']], on='product_id')
category_reviews = category_reviews.merge(reviews[['order_id', 'review_score']], on='order_id')

category_satisfaction = category_reviews.groupby('product_category_name_english').agg({
    'review_score': ['mean', 'count']
}).reset_index()
category_satisfaction.columns = ['category', 'avg_review_score', 'review_count']
category_satisfaction = category_satisfaction[category_satisfaction['review_count'] >= 30]  # Filter for categories with sufficient reviews
category_satisfaction = category_satisfaction.sort_values('avg_review_score')

# Visualize category satisfaction
plt.figure(figsize=(14, 8))
sns.barplot(x='avg_review_score', y='category', data=category_satisfaction, palette='viridis')
plt.title('Average Review Score by Product Category')
plt.xlabel('Average Review Score')
plt.ylabel('Category')
plt.axvline(reviews['review_score'].mean(), color='red', linestyle='--', label=f'Overall Average: {reviews["review_score"].mean():.2f}')
plt.legend()
plt.tight_layout()
plt.show()

print("Customer satisfaction analysis complete.")

## 9. Payment Analysis

Let's analyze payment methods and their relationship with order values.

In [ ]:
# Payment method distribution
payment_type_counts = payments['payment_type'].value_counts().reset_index()
payment_type_counts.columns = ['payment_type', 'count']

# Calculate payment statistics by method
payment_stats = payments.groupby('payment_type').agg({
    'payment_value': ['mean', 'median', 'min', 'max', 'sum', 'count']
}).reset_index()
payment_stats.columns = ['payment_type', 'avg_value', 'median_value', 'min_value', 'max_value', 'total_value', 'count']
payment_stats['share_of_total'] = payment_stats['total_value'] / payment_stats['total_value'].sum() * 100

print("Payment statistics by method:")
display(payment_stats.sort_values('total_value', ascending=False))

# Visualize payment method distribution
plt.figure(figsize=(12, 10))

plt.subplot(2, 1, 1)
sns.barplot(x='payment_type', y='count', data=payment_type_counts, palette='viridis')
plt.title('Payment Method Distribution')
plt.xlabel('Payment Type')
plt.ylabel('Count')
plt.xticks(rotation=0)

plt.subplot(2, 1, 2)
plt.pie(payment_stats['total_value'], labels=payment_stats['payment_type'], autopct='%1.1f%%', 
        startangle=90, shadow=True, explode=[0.05] * len(payment_stats))
plt.title('Share of Total Payment Value by Method')
plt.axis('equal')

plt.tight_layout()
plt.show()

# Analyze payment value distribution
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(payments['payment_value'], bins=50, kde=True)
plt.title('Payment Value Distribution')
plt.xlabel('Payment Value (BRL)')
plt.ylabel('Frequency')
plt.xlim(0, payments['payment_value'].quantile(0.95))  # Limit to 95th percentile to handle outliers

plt.subplot(1, 2, 2)
sns.boxplot(x='payment_type', y='payment_value', data=payments)
plt.title('Payment Value by Method')
plt.xlabel('Payment Type')
plt.ylabel('Payment Value (BRL)')
plt.ylim(0, payments['payment_value'].quantile(0.95))  # Limit to 95th percentile to handle outliers
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Analyze installments
installment_counts = payments['payment_installments'].value_counts().sort_index().reset_index()
installment_counts.columns = ['installments', 'count']

# Calculate average order value by installment count
installment_stats = payments.groupby('payment_installments').agg({
    'payment_value': 'mean',
    'order_id': 'count'
}).reset_index()
installment_stats.columns = ['installments', 'avg_order_value', 'order_count']

# Visualize installment distribution
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.barplot(x='installments', y='count', data=installment_counts.head(10), palette='viridis')
plt.title('Distribution of Payment Installments')
plt.xlabel('Number of Installments')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
sns.lineplot(x='installments', y='avg_order_value', data=installment_stats.head(10), marker='o')
plt.title('Average Order Value by Installment Count')
plt.xlabel('Number of Installments')
plt.ylabel('Average Order Value (BRL)')
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Analyze payment method preferences by state
payment_state = payments.merge(orders[['order_id', 'customer_id']], on='order_id')
payment_state = payment_state.merge(customers[['customer_id', 'customer_state']], on='customer_id')

# Calculate payment method share by state
payment_method_by_state = payment_state.groupby(['customer_state', 'payment_type']).size().reset_index(name='count')
payment_method_by_state['total_by_state'] = payment_method_by_state.groupby('customer_state')['count'].transform('sum')
payment_method_by_state['share'] = payment_method_by_state['count'] / payment_method_by_state['total_by_state'] * 100

# Get top 5 states by order count
top_states = payment_state['customer_state'].value_counts().head(5).index.tolist()
payment_method_by_top_states = payment_method_by_state[payment_method_by_state['customer_state'].isin(top_states)]

# Visualize payment method preferences by state
plt.figure(figsize=(14, 8))
sns.barplot(x='customer_state', y='share', hue='payment_type', data=payment_method_by_top_states, palette='viridis')
plt.title('Payment Method Preferences by State (Top 5 States)')
plt.xlabel('State')
plt.ylabel('Share (%)')
plt.legend(title='Payment Type')
plt.tight_layout()
plt.show()

print("Payment analysis complete.")

## 10. Seller Performance Analysis

Let's analyze seller performance and identify top performers.

In [ ]:
# Calculate seller performance metrics
seller_performance = order_items.merge(orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'delivery_time', 'delivery_delay']], on='order_id')
seller_performance = seller_performance.merge(reviews[['order_id', 'review_score']], on='order_id')

# Aggregate metrics by seller
seller_metrics = seller_performance.groupby('seller_id').agg({
    'order_id': 'count',
    'price': 'sum',
    'freight_value': 'sum',
    'review_score': 'mean',
    'delivery_time': 'mean',
    'delivery_delay': 'mean'
}).reset_index()

seller_metrics.columns = ['seller_id', 'order_count', 'total_sales', 'total_freight', 'avg_review_score', 'avg_delivery_time', 'avg_delivery_delay']
seller_metrics['avg_order_value'] = seller_metrics['total_sales'] / seller_metrics['order_count']

# Add seller state information
seller_metrics = seller_metrics.merge(sellers[['seller_id', 'seller_state', 'seller_city']], on='seller_id')

# Identify top sellers by sales
top_sellers_by_sales = seller_metrics.sort_values('total_sales', ascending=False).head(20)

print("Top 20 sellers by total sales:")
display(top_sellers_by_sales[['seller_id', 'seller_state', 'order_count', 'total_sales', 'avg_review_score']])

# Visualize seller performance distribution
plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
sns.histplot(seller_metrics['order_count'], bins=30, kde=True)
plt.title('Distribution of Order Count per Seller')
plt.xlabel('Order Count')
plt.ylabel('Number of Sellers')
plt.xlim(0, seller_metrics['order_count'].quantile(0.95))  # Limit to 95th percentile

plt.subplot(2, 2, 2)
sns.histplot(seller_metrics['total_sales'], bins=30, kde=True)
plt.title('Distribution of Total Sales per Seller')
plt.xlabel('Total Sales (BRL)')
plt.ylabel('Number of Sellers')
plt.xlim(0, seller_metrics['total_sales'].quantile(0.95))  # Limit to 95th percentile

plt.subplot(2, 2, 3)
sns.histplot(seller_metrics['avg_review_score'], bins=30, kde=True)
plt.title('Distribution of Average Review Score per Seller')
plt.xlabel('Average Review Score')
plt.ylabel('Number of Sellers')

plt.subplot(2, 2, 4)
sns.histplot(seller_metrics['avg_delivery_time'], bins=30, kde=True)
plt.title('Distribution of Average Delivery Time per Seller')
plt.xlabel('Average Delivery Time (days)')
plt.ylabel('Number of Sellers')
plt.xlim(0, seller_metrics['avg_delivery_time'].quantile(0.95))  # Limit to 95th percentile

plt.tight_layout()
plt.show()

# Create a scatter plot of sales vs. review score
plt.figure(figsize=(12, 8))
plt.scatter(
    seller_metrics['total_sales'], 
    seller_metrics['avg_review_score'],
    s=seller_metrics['order_count'] / 5,  # Size represents order count
    alpha=0.6
)

# Add labels for top sellers
for i, row in top_sellers_by_sales.head(10).iterrows():
    plt.annotate(
        f"Seller {row['seller_id'][:5]}...",
        xy=(row['total_sales'], row['avg_review_score']),
        xytext=(5, 5),
        textcoords='offset points'
    )

plt.title('Seller Performance: Sales vs. Review Score')
plt.xlabel('Total Sales (BRL)')
plt.ylabel('Average Review Score')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Analyze seller concentration
seller_metrics['sales_percentile'] = pd.qcut(seller_metrics['total_sales'], 10, labels=False)
seller_concentration = seller_metrics.groupby('sales_percentile').agg({
    'seller_id': 'count',
    'total_sales': 'sum'
}).reset_index()

seller_concentration['seller_percentage'] = seller_concentration['seller_id'] / seller_concentration['seller_id'].sum() * 100
seller_concentration['sales_percentage'] = seller_concentration['total_sales'] / seller_concentration['total_sales'].sum() * 100
seller_concentration['cumulative_sales_percentage'] = seller_concentration['sales_percentage'].cumsum()

print("Seller concentration analysis:")
display(seller_concentration)

# Visualize seller concentration (Pareto principle)
plt.figure(figsize=(12, 6))
plt.bar(seller_concentration['sales_percentile'], seller_concentration['sales_percentage'], alpha=0.7)
plt.plot(seller_concentration['sales_percentile'], seller_concentration['cumulative_sales_percentage'], 'ro-', linewidth=2)
plt.title('Seller Concentration: Pareto Analysis')
plt.xlabel('Seller Decile (by Sales)')
plt.ylabel('Percentage of Total Sales')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(10))
plt.tight_layout()
plt.show()

print("Seller performance analysis complete.")

## 11. Cohort Analysis

Let's perform a cohort analysis to understand customer retention over time.

In [ ]:
# Create customer cohorts based on first purchase date
customer_first_purchase = orders.groupby('customer_id')['order_purchase_timestamp'].min().reset_index()
customer_first_purchase['cohort'] = customer_first_purchase['order_purchase_timestamp'].dt.to_period('M')

# Get all customer orders with cohort information
cohort_data = orders.merge(customer_first_purchase[['customer_id', 'cohort']], on='customer_id')
cohort_data['order_month'] = cohort_data['order_purchase_timestamp'].dt.to_period('M')

# Calculate the period number (months since first purchase)
cohort_data['period_number'] = (cohort_data['order_month'].astype(str).astype('datetime64[ns]') - 
                               cohort_data['cohort'].astype(str).astype('datetime64[ns]')).dt.days // 30

# Count unique customers by cohort and period
cohort_counts = cohort_data.groupby(['cohort', 'period_number'])['customer_id'].nunique().reset_index()

# Get the total number of customers in each cohort
cohort_sizes = cohort_data.groupby('cohort')['customer_id'].nunique().reset_index()
cohort_sizes.columns = ['cohort', 'cohort_size']

# Calculate retention rate
cohort_retention = cohort_counts.merge(cohort_sizes, on='cohort')
cohort_retention['retention_rate'] = cohort_retention['customer_id'] / cohort_retention['cohort_size'] * 100

# Create a pivot table for the retention matrix
retention_matrix = cohort_retention.pivot_table(index='cohort', columns='period_number', values='retention_rate')

# Visualize the retention matrix as a heatmap
plt.figure(figsize=(15, 8))
sns.heatmap(retention_matrix, annot=True, fmt='.1f', cmap='viridis', linewidths=.5)
plt.title('Customer Cohort Retention Analysis (% of Customers Returning)')
plt.xlabel('Months Since First Purchase')
plt.ylabel('Cohort (First Purchase Month)')
plt.tight_layout()
plt.show()

# Calculate average retention by period
avg_retention = cohort_retention.groupby('period_number')['retention_rate'].mean().reset_index()

# Visualize average retention by period
plt.figure(figsize=(12, 6))
sns.lineplot(x='period_number', y='retention_rate', data=avg_retention, marker='o', linewidth=2)
plt.title('Average Customer Retention by Month')
plt.xlabel('Months Since First Purchase')
plt.ylabel('Retention Rate (%)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(avg_retention['period_number'])
plt.tight_layout()
plt.show()

print("Cohort analysis complete.")

## 12. Key Insights and Recommendations

Based on our comprehensive analysis of the Brazilian E-Commerce dataset, here are the key insights and recommendations:

### Customer Behavior Insights:

1. **Customer Segmentation**: We identified distinct customer segments including Champions, Loyal Customers, and At-Risk customers. The RFM analysis shows that a small percentage of customers contribute to a large portion of revenue.
2. **Geographic Distribution**: Orders are concentrated in certain states, particularly in the southeast region of Brazil. SP (São Paulo) has the highest number of customers and orders.
3. **Purchasing Patterns**: Most orders contain only one item, suggesting customers typically make single-item purchases rather than bundling multiple products.
4. **Payment Preferences**: Credit card is the dominant payment method, but there's significant usage of boleto (a Brazilian payment method). Higher-value orders tend to use more installments.


### Product Insights:

1. **Category Performance**: Certain product categories like furniture, electronics, and housewares dominate sales. However, some niche categories show high average order values despite lower volumes.
2. **Price Sensitivity**: The price distribution analysis shows most products fall within a specific price range, with a long tail of higher-priced items.
3. **Product Associations**: Our market basket analysis revealed interesting product combinations that are frequently purchased together, providing opportunities for cross-selling.


### Operational Insights:

1. **Delivery Performance**: There's a clear correlation between delivery delays and lower customer satisfaction. Orders delivered ahead of schedule receive significantly higher ratings.
2. **Seller Performance**: There's high variability in seller performance, with a small percentage of sellers accounting for a large portion of sales (Pareto principle).
3. **Seasonal Trends**: The time series analysis revealed clear seasonal patterns in order volume, with peaks during certain months and specific days of the week.


### Recommendations:

1. **Customer Retention Strategies**:

1. Implement targeted marketing campaigns for different customer segments
2. Develop loyalty programs for Champions and Loyal Customers
3. Create win-back campaigns for At-Risk customers



2. **Product Portfolio Optimization**:

1. Focus on expanding high-performing categories
2. Consider bundling frequently co-purchased items
3. Review pricing strategies for underperforming categories



3. **Operational Improvements**:

1. Optimize delivery processes to reduce delays
2. Implement stricter seller performance standards
3. Adjust inventory and staffing based on seasonal trends



4. **Geographic Expansion**:

1. Target marketing efforts in high-potential states
2. Consider logistics improvements in states with longer delivery times
3. Analyze the customer-to-seller ratio to identify market opportunities



5. **Payment and Financing Options**:

1. Continue offering diverse payment methods
2. Consider optimizing installment options for higher-value purchases
3. Analyze payment method preferences by region for targeted offerings